<a href="https://colab.research.google.com/github/shaaagri/iat481-nlp-proj/blob/main/LLama2_vanilla_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
In this preparation notebook, we will set up a suitable variant of Llama2 LLM to be able to do some raw queries. We then will try to integrate it into a LangChain-based pipeline and test it by asking the model some simple questions (a "vanilla" chatbot, missing yet any customization). This will help us to get ready for our project's next iteration (in another notebok) in which we will take this pipeline one step further by implementing RAG (Retrieval Augmented Generation).

*Note*: This notebook is partially based on `Chatbot_LLama_2.ipynb` from Maryiam's tutorials and also heavily borrows from `Run LLama-2 on Google Colab` tutorial by Muhammad Moin ([link](https://github.com/MuhammadMoinFaisal/LargeLanguageModelsProjects/blob/main/Run%20Llama2%20Google%20Colab/Llama_2_updated.ipynb])).

# Workflow

1. Gathering the Dependencies
2. Downloading and Loading the Model
3. Testing the Model with Raw Prompting
4. Testing the Model with LangChain

# Gathering the Dependencies


`llama.cpp`'s objective is to run the LLaMA model with 4-bit integer quantization on MacBook. It is a plain C/C++ implementation optimized for Apple silicon and x86 architectures, supporting various integer quantization and BLAS libraries. Originally a web chat example, it now serves as a development playground for ggml library features.

`GGML`, a C library for machine learning, facilitates the distribution of large language models (LLMs). It utilizes quantization to enable efficient LLM execution on consumer hardware. GGML files contain binary-encoded data, including version number, hyperparameters, vocabulary, and weights. The vocabulary comprises tokens for language generation, while the weights determine the LLM's size. Quantization reduces precision to optimize resource usage (note: in the latest versions of llama-cpp-python the GGML format was replaced with `GGUF`, meaning we will need to adjust the code provided in the examples).



## Quantized Models from the Hugging Face Community

The Hugging Face community provides quantized models, which allow us to efficiently and effectively utilize the model on the T4 GPU. It is important to consult reliable sources before using any model.

There are several variations available, but the ones that interest us are based on the GGUF library by TheBloke.

We can see the different variations that Llama-2 GGUF has [here](https://huggingface.co/models?search=llama%202%20gguf).

In this case, we will use the model called [Llama-2-13B-Chat-GGUF](https://huggingface.co/TheBloke/Llama-2-13B-Chat-GGUF).


## Installing the Packages

In [5]:
# GPU llama-cpp-python
%set_env CMAKE_ARGS="-DLLAMA_CUBLAS=on"
%set_env FORCE_CMAKE=1
!pip install llama-cpp-python --upgrade --verbose
!pip install huggingface_hub
!pip install llama-cpp-python

env: CMAKE_ARGS="-DLLAMA_CUBLAS=on"
env: FORCE_CMAKE=1
Using pip 24.0 from C:\Program Files\Python312\Lib\site-packages\pip (python 3.12)
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


## Importing the libraries


In [9]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

# Downloading and Loading the Model

Now that we have the required packages and libraries installed and imported, we can proceed to downloading and saving a quantized version of Llama-2-13b locally.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
models_root = '/content/drive/MyDrive/IAT481/481 Team Projs/NLP Project/Models/'

In [ ]:
%cd $models_root

/content/drive/.shortcut-targets-by-id/13FaXzyfvSXh_PD6h92caiYAKqKbPsFTR/481 Team Projs/NLP Project/Models


In [18]:
model_name_or_path = "TheBloke/Llama-2-7B-chat-GGUF"
model_basename = "llama-2-7b-chat.Q4_K_M.gguf"

# 13b model seems to work just slightly slower than 7b, but its scores are better across all categories
# The inference speed is fine on Colab and we thought of it as final choice, however it is too slow when run locally (on one NVIDIA RTX 4070 with 8GB VRAM)
#model_name_or_path = "TheBloke/Llama-2-13B-chat-GGUF"
#model_basename = "llama-2-13b-chat.Q4_K_M.gguf"

In [19]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

llama-2-7b-chat.Q4_K_M.gguf:   0%|          | 0.00/4.08G [00:00<?, ?B/s]

The model should be available from now locally on our google drive (turns out, it gets saved outside of it at a cache dir, but anyway). Let's try to load it into our Colab's T4 GPU and see the stats.

In [20]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=-1 # Change this value based on your model and your GPU VRAM pool, -1 tries to move all layers to GPU
    )

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from C:\Users\Narratic-DEV002\.cache\huggingface\hub\models--TheBloke--Llama-2-7B-chat-GGUF\snapshots\191239b3e26b2882fb562ffccdd1cf0f65402adb\llama-2-7b-chat.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llam

Everything seems to be okay and we are lucky that, thanks to quantization, we are able to use this LLM right from our Colab notebook, for free.

# Testing the Model with Raw Prompting

First, let's construct the default prompt template that contains a simple system prompt:

In [31]:
prompt_template=f'''SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully as possible.

  USER: %s

  ASSISTANT:
  '''

Now, without further ado, let's query the model with something and see what it gives us:

In [32]:
question='Where Simon Fraser University is located?'

In [33]:
response=lcpp_llm(prompt=prompt_template % question, max_tokens=256, temperature=0.4, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)

print(response)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2754.25 ms
llama_print_timings:      sample time =      18.35 ms /   104 runs   (    0.18 ms per token,  5667.88 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =   17143.92 ms /   104 runs   (  164.85 ms per token,     6.07 tokens per second)
llama_print_timings:       total time =   17411.91 ms /   105 tokens


{'id': 'cmpl-9684be21-405e-470d-ad16-7c8d05ab07af', 'object': 'text_completion', 'created': 1712806907, 'model': 'C:\\Users\\Narratic-DEV002\\.cache\\huggingface\\hub\\models--TheBloke--Llama-2-7B-chat-GGUF\\snapshots\\191239b3e26b2882fb562ffccdd1cf0f65402adb\\llama-2-7b-chat.Q4_K_M.gguf', 'choices': [{'text': 'SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully as possible.\n\n  USER: Where Simon Fraser University is located?\n\n  ASSISTANT:\n   Simon Fraser University (SFU) is located in British Columbia, Canada. Specifically, it is situated on the unceded traditional territories of the Coast Salish peoples, including the Musqueam, Squamish and Tsleil-Waututh nations. The main campus of SFU is located in Burnaby, a city just outside of Vancouver. Additionally, SFU has satellite campuses in Surrey and Richmond, as well as several other locations throughout British Columbia.', 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_

Great! We get a sensible response from Llama-2 to our prompt which means we have done our first LLM setup successfully. 

**Warning:** from this point onwards we had to abandon Google Colab and set up a local Jupyter Lab environment (Windows, RTX 4070) and continue the work, as we had hit Colab usage limits, unfortunately. However, we hope those who wish to test it in Colab, where this notebook was supposed to be run, can that without any issues since Colab and Jupyter are designed to be interchangeable.

# Testing the Model with LangChain

Now that we have managed to get Llama-2 working in its raw form, let's try to integrate it into a trivial LangChain pipeline and use it instead to prompt the model.

Setting it up is not very hard, thanks to detailed documentation at the LangChain website ([link](https://python.langchain.com/docs/integrations/llms/llamacpp/)).

In [29]:
!pip install langchain

Defaulting to user installation because normal site-packages is not writeable
  Using cached SQLAlchemy-2.0.29-cp312-cp312-win_amd64.whl.metadata (9.8 kB)
     ---------------------------------------- 0.0/85.1 kB ? eta -:--:--
     ---------------------------------------- 85.1/85.1 kB 2.4 MB/s eta 0:00:00
     ---------------------------------------- 0.0/50.7 kB ? eta -:--:--
     ---------------------------------------- 50.7/50.7 kB ? eta 0:00:00
  Using cached greenlet-3.0.3-cp312-cp312-win_amd64.whl.metadata (3.9 kB)
   ---------------------------------------- 0.0/814.5 kB ? eta -:--:--
   --------------------------------------  809.0/814.5 kB 17.0 MB/s eta 0:00:01
   --------------------------------------- 814.5/814.5 kB 12.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/363.4 kB ? eta -:--:--
   --------------------------------------- 363.4/363.4 kB 22.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   --------------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [59]:
from langchain_community.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain_core.prompts import PromptTemplate

In [60]:
prompt = PromptTemplate.from_template(prompt_template)

In [61]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [76]:
llm = LlamaCpp(
    # Make sure the model path is correct for your system!
    model_path="/Users/Narratic-DEV002/.cache/huggingface/hub/models--TheBloke--Llama-2-7B-chat-GGUF/snapshots/191239b3e26b2882fb562ffccdd1cf0f65402adb/llama-2-7b-chat.Q4_K_M.gguf",
    
    temperature=0.4,
    max_tokens=1024,
    repeat_penalty=1.2,
    #top_p=1,
    top_k=150,
    callback_manager=callback_manager,
    verbose=True,  # Verbose is required to pass to the callback manager
)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /Users/Narratic-DEV002/.cache/huggingface/hub/models--TheBloke--Llama-2-7B-chat-GGUF/snapshots/191239b3e26b2882fb562ffccdd1cf0f65402adb/llama-2-7b-chat.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.

In [66]:
print(question)

Where Simon Fraser University is located?


In [82]:
llm.invoke(question)

Llama.generate: prefix-match hit


Where is the campus of Simon Fraser University located?
Simon Fraser University (SFU) is a public research university located in British Columbia, Canada. The main campus of SFU is situated in Burnaby, which is a city located in the Lower Mainland region of Greater Vancouver. Specifically, the campus is located at 8888 University Drive, Burnaby, BC V5A 1S6.
The Simon Fraser University campus spans over 30 acres and features several buildings, including academic facilities, residence halls, recreation centers, and student services. The university also has satellite campuses in Surrey and Vancouver, as well as a research station located on the coast of Burrard Inlet in North Vancouver.
Overall, Simon Fraser University is easily accessible by car or public transportation, with convenient connections to downtown Vancouver and other nearby cities.


llama_print_timings:        load time =     551.45 ms
llama_print_timings:      sample time =      26.64 ms /   198 runs   (    0.13 ms per token,  7431.32 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =   17995.64 ms /   198 runs   (   90.89 ms per token,    11.00 tokens per second)
llama_print_timings:       total time =   18630.90 ms /   199 tokens


'\nWhere is the campus of Simon Fraser University located?\nSimon Fraser University (SFU) is a public research university located in British Columbia, Canada. The main campus of SFU is situated in Burnaby, which is a city located in the Lower Mainland region of Greater Vancouver. Specifically, the campus is located at 8888 University Drive, Burnaby, BC V5A 1S6.\nThe Simon Fraser University campus spans over 30 acres and features several buildings, including academic facilities, residence halls, recreation centers, and student services. The university also has satellite campuses in Surrey and Vancouver, as well as a research station located on the coast of Burrard Inlet in North Vancouver.\nOverall, Simon Fraser University is easily accessible by car or public transportation, with convenient connections to downtown Vancouver and other nearby cities.'

Awesome! We got Llama-2 working within a LangChain pipeline, which means we are now equipped to up the game by inserting a vector database, a retriever, and a simple UI into the pipeline, to build a RAG-based chatbot as we have conceived. The response, honestly, looks good, at the level of ChatGPT, which is quite amazing given the model is quantized and being run locally.

A bit odd behavior we noted happens when you repeat the same question to the model - sometimes it can diverge and, instead of providing a sensible answer like above, start coming up with additional irrelevant questions to itself, polluting the response. This does not happen with the raw prompting, so this may be due to LangChain doing something behind the scenes. Also, this issue does not seem to happen if the model is re-imported in Cell 68. We will keep an eye on it during the next steps.

Lastly, we have noticed the initial parameters of the LlamaCpp constructor (see Cell 68) carry quite an influence on the quality of responses ([link to the full list of the params at LangChain documentation](https://api.python.langchain.com/en/latest/llms/langchain_community.llms.llamacpp.LlamaCpp.html)). The ones we are using right now seem to be optimal, but we will keep an eye on them as well as a way to control how well Llama-2's behavior matches our expectations.